In [1]:
import json
import os
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

In [2]:
temp_seoul_store = pd.read_csv("seoul_store_info_202106.csv")
temp_seoul_store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316078 entries, 0 to 316077
Data columns (total 39 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   상가업소번호     316078 non-null  int64  
 1   상호명        316077 non-null  object 
 2   지점명        54615 non-null   object 
 3   상권업종대분류코드  316078 non-null  object 
 4   상권업종대분류명   316078 non-null  object 
 5   상권업종중분류코드  316078 non-null  object 
 6   상권업종중분류명   316078 non-null  object 
 7   상권업종소분류코드  316078 non-null  object 
 8   상권업종소분류명   316078 non-null  object 
 9   표준산업분류코드   286026 non-null  object 
 10  표준산업분류명    286026 non-null  object 
 11  시도코드       316078 non-null  int64  
 12  시도명        316078 non-null  object 
 13  시군구코드      316078 non-null  int64  
 14  시군구명       316078 non-null  object 
 15  행정동코드      316078 non-null  int64  
 16  행정동명       315898 non-null  object 
 17  법정동코드      316078 non-null  int64  
 18  법정동명       316078 non-null  object 
 19  지번코드       316078 non-n

In [3]:
temp_seoul_store.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010100108540018009586,상랑의빌딩,서울특별시 서초구 동광로18길 82,137837,6572.0,NaN,1,NaN,126.991394,37.488375
1,17174119,쓰리에프,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1159010700101390073009536,NaN,서울특별시 동작구 동작대로27가길 12,156816,7008.0,NaN,NaN,NaN,126.980952,37.487105
2,17174096,채움,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1111012400100580000017956,두산위브파빌리온,서울특별시 종로구 삼봉로 81,110858,3150.0,NaN,NaN,NaN,126.981794,37.572387
3,17174062,호구의주방,NaN,D,소매,D07,가정/주방/인테리어,D07A17,주방가구판매,G47520,...,1156012800101600002019768,약산상가아파트,서울특별시 영등포구 선유로 269,150867,7206.0,NaN,NaN,NaN,126.897892,37.536700
4,17174040,다향,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1165010800114850004022127,NaN,서울특별시 서초구 효령로 230,137869,6709.0,NaN,1,NaN,127.009382,37.483436


In [5]:
seoul_cafe_store = temp_seoul_store[temp_seoul_store['상권업종소분류코드'] == 'Q12A01']
seoul_cafe_store.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
28,20659878,커피빈,코리아대학로대명거리점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1111017200100510000032165,NaN,서울특별시 종로구 대명길 25,110524,3078.0,NaN,3,NaN,127.000171,37.583149
111,20676428,요거프레소,쌍문점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1132010500100200000006625,NaN,서울특별시 도봉구 도당로 7,132030,1385.0,NaN,1,NaN,127.034746,37.658728
190,20695531,메머드커피,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1144012000103270015009846,NaN,서울특별시 마포구 와우산로 149,121836,4052.0,NaN,1,NaN,126.929692,37.554599
197,28491095,우성커피숍,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1147010300105140002010707,NaN,서울특별시 양천구 오목로 48,158090,8029.0,NaN,1,NaN,126.842935,37.522055
225,20815616,카페클럽,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1171010800100430006004404,NaN,서울특별시 송파구 동남로2길 3-6,138200,5805.0,NaN,1,NaN,127.122480,37.488907


In [8]:
seoul_cafe_location = [[row["위도"], row["경도"]] for _, row in seoul_cafe_store.iterrows()]

In [9]:
bus_line_info = pd.read_csv("seoul_bus_line_info.csv")
bus_line_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39363 entries, 0 to 39362
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   노선ID    39363 non-null  int64  
 1   노선명     39363 non-null  object 
 2   순번      39363 non-null  int64  
 3   구간ID    39363 non-null  int64  
 4   정류소ID   39363 non-null  int64  
 5   ARS-ID  39363 non-null  int64  
 6   정류소명    39363 non-null  object 
 7   X좌표     39363 non-null  float64
 8   Y좌표     39363 non-null  float64
dtypes: float64(2), int64(5), object(2)
memory usage: 2.7+ MB


In [10]:
bus_line_info.head()

,노선ID,노선명,순번,구간ID,정류소ID,ARS-ID,정류소명,X좌표,Y좌표
0,100100124,0017,1,0,102000271,3689,청암자이아파트,126.946517,37.534363
1,100100124,0017,2,102700549,102000204,3298,청암동강변삼성아파트,126.949304,37.533961
2,100100124,0017,3,102700550,102000227,3321,청심경로당,126.950449,37.533744
3,100100124,0017,4,102700551,102000210,3304,원효2동주민센터,126.950904,37.534278
4,100100124,0017,5,102700552,102000212,3306,산천동,126.953984,37.535420


In [11]:
bus_line_info.rename(columns={"Y좌표": "lat","X좌표": "lng",}, inplace=True)
bus_line_info.head()

,노선ID,노선명,순번,구간ID,정류소ID,ARS-ID,정류소명,lng,lat
0,100100124,0017,1,0,102000271,3689,청암자이아파트,126.946517,37.534363
1,100100124,0017,2,102700549,102000204,3298,청암동강변삼성아파트,126.949304,37.533961
2,100100124,0017,3,102700550,102000227,3321,청심경로당,126.950449,37.533744
3,100100124,0017,4,102700551,102000210,3304,원효2동주민센터,126.950904,37.534278
4,100100124,0017,5,102700552,102000212,3306,산천동,126.953984,37.535420


In [13]:
bus_station_location = [[bus_line_info.loc[index_num, 'lat'], bus_line_info.loc[index_num, 'lng']] for index_num in range(len(bus_line_info))]

In [14]:
center = [37.541, 126.986]
map = folium.Map(
    location=center,
    tiles="Stamen Terrain",
    zoom_start=15
)

In [15]:
fg = folium.FeatureGroup(name="groups")
map.add_child(fg)
cafe = plugins.FeatureGroupSubGroup(fg, "cafe_density")
map.add_child(cafe)
bus = plugins.FeatureGroupSubGroup(fg, "bus_station_density")
map.add_child(bus)
folium.LayerControl(collapsed=False).add_to(map)

In [16]:
plugins.HeatMap(seoul_cafe_location).add_to(cafe)
plugins.HeatMap(bus_station_location).add_to(bus)

In [17]:
map